In [2]:
pip install psycopg[binary]


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import psycopg
import pandas as pd
from psycopg.rows import dict_row
import os

testfligoo_connection = os.environ['medicaldata_connection']

with psycopg.connect(testfligoo_connection, row_factory=dict_row) as conn:

    # Open a cursor to perform database operations
    with conn.cursor() as cur:
        try:
            # Query the database and obtain data as Python objects.
            cur.execute("SELECT * FROM stay_daily_cost")
            result = cur.fetchall()
            
            data = pd.DataFrame(result)
            display(data)

        
        except BaseException:
            conn.rollback()
        
        else:
            conn.commit()
        
        finally:
            conn.close()


""
